In [8]:
from scipy.stats import norm
import pandas as pd
import numpy as np
import sys, os

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

In [9]:
def get_d2(options):

    if len(options) == 0:
        return options

    o = options.copy()
    m = o.option_type.map({"C" : 1, "P" : -1}).values

    tau = o.days_to_expiry.values / 365
    rtau = np.sqrt(tau)
    iv = o.implied_volatility.values / 100
    S = o.stock_price.values
    K = o.strike_price.values
    q = o.dividend_yield.values / 100
    q = np.log(1 + q)
    r = o.rate.values / 100
    r = np.log(1 + r)

    ###################################################################################################

    eqt = np.exp(-q * tau)
    kert = K * np.exp(-r * tau)

    d1 = np.log(S / K)
    d1 += (r - q + 0.5 * (iv ** 2)) * tau
    d1 /= iv * rtau
    d2 = d1 - iv * rtau

    return d2

### Data

In [10]:
options = pd.read_csv("data/spy_options.csv")
fridays = pd.date_range("2020-01-01", "2025-01-01", freq="WOM-3FRI").astype(str)
thursdays = pd.date_range("2020-01-01", "2025-01-01", freq="WOM-3THU").astype(str)
regulars = list(fridays) + list(thursdays)
options = options[options.expiration_date.isin(regulars)]

### Calc D2

In [11]:
options['d2'] = get_d2(options)
options['cnd2'] = norm.cdf(options.d2)
options['pnd2'] = norm.cdf(-options.d2)
options.shape

/home/zquantz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in true_divide


(4533, 32)

### Filter Bad Entries

In [12]:
options = options[options.bid_price != 0]
options = options[options.ask_price != 0]
options = options[options.implied_volatility != 0]
options = options[abs(options.strike_price / options.stock_price - 1) <= 0.40]

### Full Code

In [14]:
x1max, x2max = 0, 0
x1min, x2min = 1000, 1000

for i in options.days_to_expiry.unique():
    
    print(i)
    tmp = options[options.days_to_expiry == i]
    c = tmp[tmp.option_type == "C"]
    p = tmp[tmp.option_type == "P"]
    
    crncdf = [
        c.cnd2.values[0] - c.cnd2.values[i]
        for i in range(1, c.shape[0])
    ]
    crncdf = pd.Series(crncdf).rolling(3, min_periods=1, win_type="triang").mean()
    crncdf /= crncdf.max()

    crncdf.name = "F(x)"
    
    prncdf = [
        p.pnd2.values[i] - p.pnd2.values[0]
        for i in range(1, p.shape[0])
    ]
    prncdf = pd.Series(prncdf).rolling(3, min_periods=1, win_type="triang").mean()
    prncdf /= prncdf.max()

    prncdf.name = "F(x)"
    
    Kc = c.strike_price.reset_index(drop=True)
    Kc.name = "Stock Price"

    Kp = p.strike_price.reset_index(drop=True)
    Kp.name = "Stock Price"
    
    x1max = max(x1max, Kc.max())
    x2max = max(x2max, Kp.max())
    
    x1min = min(x1min, Kc.min())
    x2min = min(x2min, Kp.min())
    
    f, ax = plt.subplots(2, 2, figsize=(17, 13))

    sns.scatterplot(Kc, crncdf, ax=ax[0, 0])
    sns.scatterplot(Kc, crncdf.diff(), ax=ax[0, 1])
    ax[0, 1].set_ylabel("f(x)")
    ax[0, 1].axvline(x=options.stock_price.values[0], color="black", alpha=0.5)

    sns.scatterplot(Kp, prncdf, ax=ax[1, 0])
    sns.scatterplot(Kp, prncdf.diff(), ax=ax[1, 1])
    ax[1, 1].set_ylabel("f(x)")
    ax[1, 1].axvline(x=options.stock_price.values[0], color="black", alpha=0.5)
    
    f.suptitle(f"{i} Day{'s' if i > 1 else ''} to Expiration")

    ax[0, 1].set_xlim(200, 500)
    ax[1, 1].set_xlim(200, 500)
    
    ax[0, 0].set_xlim(200, 500)
    ax[1, 0].set_xlim(200, 500)
    
    ax[0, 1].set_ylim(0, 0.15)
    ax[1, 1].set_ylim(0, 0.15)
    
    ax[0, 0].set_ylim(0, 1.05)
    ax[1, 0].set_ylim(0, 1.05)
    
    ax[0, 0].set_title("Call Implied CDF")
    ax[0, 1].set_title("Call Implied Density")
    ax[1, 0].set_title("Put Implied CDF")
    ax[1, 1].set_title("Put Implied Density")
    
    plt.savefig(f"plots/rnd_{i}.png")
    plt.close()

3
31
59
94
122
150
185
213
304
395
430
486
577
668
759
794
